In [4]:
import os

print(os.getcwd())


C:\gh\snu-amc-s19-team7\Server


In [5]:
import asyncio
from TestClient import test_client

from sidecar import Sidecar
from ipyleaflet import *

snu_position = (37.4556699,126.9533264)
num_clients = 15

m = Map(center=snu_position, zoom=15, basemap=basemaps.Esri.WorldStreetMap)

markers = []
for i in range(num_clients):
    markers.append(Marker(location=m.center))
    m += markers[i]

sc = Sidecar(title='Sidecar Output')
with sc:
    display(m)
    
def client_callback(clients):
    for i in range(num_clients):
        markers[i].center = clients[i].position

await test_client.execute(asyncio.get_running_loop(), test_client.RoleUpdateTest, num_clients, 'http://localhost:8080/', client_callback)

SyntaxError: import * only allowed at module level (<ipython-input-5-8f910c076247>, line 2)